## Convertisseur UTM Nord Sahara ⇆ WGS84

**Français** : Ce carnet décrit comment le convertisseur passe de coordonnées géographiques (WGS84) aux coordonnées UTM Nord Sahara (zones 29–32, k0=0.9996, a=6378249.138, b=6356514.9999) et inversement. Il montre aussi la création d’un KML fermé prêt pour Google Earth.

**العربية** : يشرح هذا الدفتر كيفية تحويل الإحداثيات الجغرافية (WGS84) إلى نظام UTM الصحراء الشمالية (المناطق 29–32، معامل المقياس 0.9996، نصف القطرين a=6378249.138 و b=6356514.9999) والعكس، مع توليد ملف KML مغلق لفتحه في Google Earth.

### Paramètres de projection
- Projection : `+proj=utm`
- Zone : 29 à 32 (Nord Algérie)
- Ellipsoïde local : a = 6378249.138, b = 6356514.9999
- Facteur d’échelle (k0) : 0.9996
- Faux Est (x0) : 500000 m, Faux Nord (y0) : 0 m

هذه القيم تضمن تطابق التحويل مع خرائط الجزائر في نظام UTM الصحراء الشمالية.

In [ ]:
from pyproj import Transformer

def utm_nord_sahara(zone:int):
    return f"+proj=utm +zone={zone} +a=6378249.138 +b=6356514.9999 +units=m +k=0.9996 +x_0=500000 +y_0=0 +no_defs"

# Exemple : WGS84 -> UTM (zone 31)
wgs84_to_utm = Transformer.from_crs("EPSG:4326", utm_nord_sahara(31), always_xy=True)
coords_wgs84 = [(-6.655214, 26.204379), (-6.652213, 26.204330)]
coords_utm = [wgs84_to_utm.transform(lon, lat) for lon, lat in coords_wgs84]
coords_utm  # (X, Y) en mètres

# Inverse : UTM -> WGS84
utm_to_wgs84 = Transformer.from_crs(utm_nord_sahara(31), "EPSG:4326", always_xy=True)
coords_back = [utm_to_wgs84.transform(x, y) for x, y in coords_utm]
coords_back

### Fermeture et export KML
- On ferme automatiquement le polygone en répétant le premier point s’il n’est pas déjà fermé.
- On produit un KML avec `<LineString>` (ou `<Polygon>` si besoin) pour Google Earth.

يتم إغلاق المضلع تلقائيًا بإعادة النقطة الأولى إذا لم يكن المغلق موجودًا، ثم إنشاء ملف KML جاهز لــ Google Earth.

In [ ]:
def close_ring(coords):
    if len(coords) < 3:
        return coords
    if coords[0] != coords[-1]:
        return coords + [coords[0]]
    return coords

def kml_from_lonlat(coords):
    ring = close_ring(coords)
    coord_str = " ".join(f"{lon},{lat},0" for lon, lat in ring)
    return f"""<?xml version='1.0' encoding='UTF-8'?>
<kml xmlns='http://www.opengis.net/kml/2.2'>
  <Document>
    <Placemark>
      <name>Export Sigam</name>
      <LineString><tessellate>1</tessellate><coordinates>{coord_str}</coordinates></LineString>
    </Placemark>
  </Document>
</kml>"""

coords_example = [(-6.655214, 26.204379), (-6.652213, 26.204330), (-6.652268, 26.201623)]
kml_text = kml_from_lonlat(coords_example)
print(kml_text[:200], "...")

### Détail du fonctionnement (FR)
1. **Saisie par colonnes X/Y ou Lon/Lat** : on sépare les valeurs dans deux champs pour éviter les erreurs de format. Chaque ligne de X est appariée à la même ligne de Y.
2. **Nettoyage & validation** : suppression des parenthèses éventuelles, contrôle que chaque valeur est numérique et que le nombre de lignes X = nombre de lignes Y.
3. **Projection** : utilisation de la chaîne UTM Nord Sahara (zone 29–32) avec `+proj=utm +zone=<Z> +a=6378249.138 +b=6356514.9999 +units=m +k=0.9996 +x_0=500000 +y_0=0 +no_defs` et WGS84 (EPSG:4326) via proj4/pyproj.
4. **Arrondi optionnel** : on peut arrondir au mètre pour retrouver des valeurs hectométriques propres (utile pour comparer à des relevés terrain).
5. **Fermeture du polygone** : si au moins 3 points et que le dernier est différent du premier, on répète le premier point pour fermer le tracé (évite les polylignes ouvertes dans Google Earth).
6. **Export KML** : génération d’un KML simple (`LineString`), téléchargeable ou ouvrable directement dans Google Earth Web. Les coordonnées sont en lon,lat,0.

### شرح مفصّل (AR)
1. **إدخال منفصل لـ X/Y أو خط الطول/العرض**: فصل القيم في حقول مستقلة لتفادي أخطاء التنسيق. يتم مطابقة كل سطر X مع السطر المقابل في Y.
2. **تنظيف والتحقق**: إزالة الأقواس إن وجدت، التحقق من أن كل القيم رقمية وأن عدد الأسطر في X يساوي عدد الأسطر في Y.
3. **الإسقاط**: استعمال سلسلة UTM الصحراء الشمالية (المناطق 29–32) مع `+proj=utm +zone=<Z> +a=6378249.138 +b=6356514.9999 +units=m +k=0.9996 +x_0=500000 +y_0=0 +no_defs` و WGS84 (EPSG:4326) عبر proj4/pyproj.
4. **تقريب اختياري إلى المتر**: يمكن التقريب لأقرب متر للحصول على قيم هكتومترية نظيفة (مفيد لمقارنة القياسات الميدانية).
5. **إغلاق المضلع**: إذا كان هناك 3 نقاط أو أكثر وكان آخر نقطة تختلف عن الأولى، يتم تكرار النقطة الأولى لإغلاق المسار (منع الخطوط المفتوحة في Google Earth).
6. **تصدير KML**: توليد ملف KML بسيط (`LineString`) قابل للتنزيل أو الفتح مباشرة في Google Earth Web. الإحداثيات بصيغة خط الطول، خط العرض، 0.